In [32]:
import praw
import pandas as pd
import time
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"  # multiple output per jupyter notebook code block

# 用你的凭证初始化PRAW
reddit = praw.Reddit(client_id='9Ry3ASf3UTPWB4p4G_Tn4Q',
                     client_secret='FzGe_N27xm5NUa0hsYGtn2x4dv6rQA',
                     username='albusmaxgranger',
                     password='ma19950903',
                     user_agent='hh')


In [33]:
subreddit_name = 'wallstreetbets'
topic = 'AI'

## Data Collection (Class, Attribute and Method)

### major datatype

In [39]:
# post
help(praw.models.reddit.submission)

Help on module praw.models.reddit.submission in praw.models.reddit:

NAME
    praw.models.reddit.submission - Provide the Submission class.

CLASSES
    builtins.object
        SubmissionFlair
    praw.models.listing.mixins.submission.SubmissionListingMixin(praw.models.base.PRAWBase)
        Submission(praw.models.listing.mixins.submission.SubmissionListingMixin, praw.models.reddit.mixins.UserContentMixin, praw.models.reddit.mixins.fullname.FullnameMixin, praw.models.reddit.base.RedditBase)
    praw.models.reddit.base.RedditBase(praw.models.base.PRAWBase)
        Submission(praw.models.listing.mixins.submission.SubmissionListingMixin, praw.models.reddit.mixins.UserContentMixin, praw.models.reddit.mixins.fullname.FullnameMixin, praw.models.reddit.base.RedditBase)
    praw.models.reddit.mixins.ThingModerationMixin(praw.models.reddit.mixins.modnote.ModNoteMixin)
        SubmissionModeration(praw.models.reddit.mixins.ThingModerationMixin, praw.models.reddit.mixins.modnote.ModNoteMixin)
   

In [38]:
# commentforrest
help(praw.models.reddit.submission.CommentForest)

Help on class CommentForest in module praw.models.comment_forest:

class CommentForest(builtins.object)
 |  CommentForest(submission: 'praw.models.Submission', comments: Optional[List[ForwardRef('praw.models.Comment')]] = None)
 |  
 |  A forest of comments starts with multiple top-level comments.
 |  
 |  Each of these comments can be a tree of replies.
 |  
 |  Methods defined here:
 |  
 |  __getitem__(self, index: int)
 |      Return the comment at position ``index`` in the list.
 |      
 |      This method is to be used like an array access, such as:
 |      
 |      .. code-block:: python
 |      
 |          first_comment = submission.comments[0]
 |      
 |      Alternatively, the presence of this method enables one to iterate over all top
 |      level comments, like so:
 |      
 |      .. code-block:: python
 |      
 |          for comment in submission.comments:
 |              print(comment.body)
 |  
 |  __init__(self, submission: 'praw.models.Submission', comments: Opt

In [40]:
# 获取指定subreddit下的帖子及其评论
def get_posts(subreddit_name, topic, limit=10):
    subreddit = reddit.subreddit(subreddit_name)
    posts = []
    for post in subreddit.search(topic,limit=limit):
        post_info = {
            # fundamental info
            'id': post.id, # ID of the submission.
            'title': post.title, # The title of the submission.
            'author': post.author_flair_text, # The text content of the author's flair, or ``None`` if not flaired.
            'created_utc': post.created_utc, # Time the submission was created, represented in `Unix Time`_.
            'time': time.strftime('%Y-%m-%d', time.localtime(post.created_utc)),
            'selftext': post.selftext, # The submissions' selftext - an empty string if a link post.
            # score related
            'distinguished': post.distinguished, # Whether or not the submission is distinguished.
            'score': post.score, # The number of upvotes for the submission.
            'upvote_ratio': post.upvote_ratio, # The percentage of upvotes from all votes on the submission.            
            'num_comments': post.num_comments, # The number of comments on the submission.
            'num_top_level_comments': post.comments.__len__(), # number of top-level comments in the forest.
            # next level
            'comments': [], # flattened list of all comments.
            # subreddit
            'subreddit_id': post.subreddit.id, # ID of the subreddit.
            'subreddit_name': post.subreddit.name, # Fullname of the subreddit.
            'subreddit_count': post.subreddit.subscribers # Count of subscribers.
        }
        while True:
            try:
                post.comments.replace_more(limit=None, threshold=0)
                break
            except:
                print("Handling replace more exception”)
                time.sleep(60)
        for comment in post.comments.list():  # Keep the top 20 comments of posts.
            comment_info = {
                # fundamental info
                'id': comment.id, # ID of the comment.
                'link_id': comment.link_id, # The submission ID that the comment belongs to.
                'created_utc': comment.created_utc, # Time the comment was created, represented in `Unix Time`_.
                'time': time.strftime('%Y-%m-%d', time.localtime(comment.created_utc)),
                'body': comment.body, # The body of the comment, as Markdown.
                # score related                
                'distinguished': comment.distinguished,
                'score': comment.score, # The number of upvotes for the comment.
                # next level
                'replies': len(comment.replies.list()),
                # subreddit
                'subreddit_id': comment.subreddit_id # The subreddit ID that the comment belongs to.
            }
            post_info['comments'].append(comment_info)
        posts.append(post_info)
    return posts



In [41]:
# 使用示例
posts = get_posts(subreddit_name,topic,limit = 2)

In [42]:
comments_data = [(post['id'], post['title'], post['time'], post['selftext'], post['score'], post['upvote_ratio'], post['num_comments'], post['num_top_level_comments'], post['subreddit_id'], comment['link_id'], comment['time'], comment['body'], comment['score'], comment['subreddit_id']) for post in posts for comment in post['comments']]
df_comments = pd.DataFrame(comments_data, columns=['Post ID', 'Post Title', 'Post Time', 'Post Selftxt', 'Post Score', 'Post Upratio', 'Post Num_c', 'Post Num_top_c', 'Post Sub_id', 'Comment L_id', 'Comment Time', 'Comment Body', 'Comment Score', 'Comment Sub_id'])

In [43]:
df_comments.dtypes

Post ID            object
Post Title         object
Post Time          object
Post Selftxt       object
Post Score          int64
Post Upratio      float64
Post Num_c          int64
Post Num_top_c      int64
Post Sub_id        object
Comment L_id       object
Comment Time       object
Comment Body       object
Comment Score       int64
Comment Sub_id     object
dtype: object

In [45]:
df_comments

,Post ID,Post Title,Post Time,Post Selftxt,Post Score,Post Upratio,Post Num_c,Post Num_top_c,Post Sub_id,Comment L_id,Comment Time,Comment Body,Comment Score,Comment Sub_id
0,1b0nau2,Ai bubble,2024-02-27,What does everyone think i mean sp500 is up al...,2354,0.95,609,230,2th52,t3_1b0nau2,2024-02-27,\n**User Report**| | | |\n:--|:--|:--|:--\n**T...,1,t5_2th52
1,1b0nau2,Ai bubble,2024-02-27,What does everyone think i mean sp500 is up al...,2354,0.95,609,230,2th52,t3_1b0nau2,2024-02-27,I believe this will be the first 24 trillion m...,2300,t5_2th52
2,1b0nau2,Ai bubble,2024-02-27,What does everyone think i mean sp500 is up al...,2354,0.95,609,230,2th52,t3_1b0nau2,2024-02-27,"I'm sure there's a bubble in there, especially...",950,t5_2th52
3,1b0nau2,Ai bubble,2024-02-27,What does everyone think i mean sp500 is up al...,2354,0.95,609,230,2th52,t3_1b0nau2,2024-02-27,"Phew thank god I saw Cathies, almost had me wo...",295,t5_2th52
4,1b0nau2,Ai bubble,2024-02-27,What does everyone think i mean sp500 is up al...,2354,0.95,609,230,2th52,t3_1b0nau2,2024-02-27,Number one rule of Wall Street: Nobody - I don...,282,t5_2th52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
793,1b1o0v1,Beyond AI,2024-02-28,,3604,0.98,218,104,2th52,t3_1b1o0v1,2024-02-28,Can you simulate it on your app?,3,t5_2th52
794,1b1o0v1,Beyond AI,2024-02-28,,3604,0.98,218,104,2th52,t3_1b1o0v1,2024-02-28,Was that sarcasm?,1,t5_2th52
795,1b1o0v1,Beyond AI,2024-02-28,,3604,0.98,218,104,2th52,t3_1b1o0v1,2024-02-28,"It's from holy grail movie, quoted with a typo...",2,t5_2th52
796,1b1o0v1,Beyond AI,2024-02-28,,3604,0.98,218,104,2th52,t3_1b1o0v1,2024-02-28,Optionsprofitcalculator.com\n\nOr if you have ...,6,t5_2th52
